<a href="https://colab.research.google.com/github/Silfa2/nlpcc-ui-2025/blob/main/week5_6_task2_tabular.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tugas Gabungan Minggu 5 & 6: Aplikasi LLM Interaktif

**Nama:** Silfa Wulandari

**NPM:** 2206090192

**Tugas 2: Pemrosesan Data Tabular dengan Dukungan LLM (65 Poin)**

## 🎯 Objektif Tugas 2
Menerapkan kapabilitas Model Bahasa Skala Besar (LLM) untuk memproses secara programatik beberapa baris data dari sumber tabular (misalnya, file CSV). Tujuannya adalah untuk melakukan tugas tertentu pada kolom teks di setiap baris, seperti merangkum, menganalisis sentimen, mengekstrak informasi, atau mengkategorikan.

## 📚 Langkah Pengerjaan
Notebook ini akan memandu langkah-langkah pengerjaan Tugas 2:
1.  **Instalasi & Impor Library**: Menyiapkan paket Python yang dibutuhkan (OpenAI, Pandas).
2.  **Pengaturan Kunci API**: Mengambil kunci API OpenAI dengan aman menggunakan Colab Secrets.
3.  **Inisialisasi Klien OpenAI**: Membuat objek klien untuk berinteraksi dengan API.
4.  **Pemilihan & Pemuatan Dataset**:
    *   Memilih dataset tabular yang sesuai (dari Kaggle atau membuat sendiri).
    *   Mengunggah dan memuat dataset menggunakan Pandas.
5.  **Definisi Fungsi Pemrosesan LLM**: Membuat fungsi untuk mengirim teks dari satu baris data ke LLM dengan *prompt* tertentu.
6.  **Iterasi & Pemrosesan Data**:
    *   Meminta pengguna memilih kolom teks yang akan diproses dan *prompt* yang akan digunakan.
    *   Melakukan iterasi pada baris-baris dataset.
    *   Mengirim data dari setiap baris ke LLM untuk diproses.
    *   Menyimpan hasil respons LLM.
7.  **Penyajian Hasil**: Menampilkan data asli bersama dengan hasil yang dihasilkan LLM.
8.  **Penjelasan & Analisis**: Detail mengenai dataset, tugas yang dilakukan, struktur *prompt*, dan analisis singkat hasil.


Mari kita mulai memproses data dengan kekuatan AI! 📊🤖



In [2]:
# Tahap 1.1: Instalasi Library yang Dibutuhkan
# Kita butuh 'openai' untuk LLM dan 'pandas' untuk mengelola data tabular.
# 'openpyxl' mungkin dibutuhkan jika Anda menggunakan file .xlsx, pandas akan memberitahu jika perlu.
!pip install openai pandas openpyxl -q

# Tahap 1.2: Impor Library
import openai
import pandas as pd
from google.colab import userdata, files # Untuk Colab Secrets dan upload file
import io # Untuk menangani byte stream dari file yang diupload
from IPython.display import display, Markdown # Untuk output yang lebih baik
import json # Untuk mem-parse output JSON dari LLM jika diperlukan

print("✅ Library OpenAI, Pandas, dan pendukung lainnya telah siap!")

# Tahap 1.3: Pengaturan Kunci API OpenAI (Sama seperti Tugas 1)
try:
    OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
    if not OPENAI_API_KEY:
        display(Markdown("⚠️ **Peringatan:** Kunci API 'openai_key' ditemukan, tetapi nilainya kosong."))
        raise ValueError("Kunci API 'openai_key' kosong.")
    print("🔑 Kunci API OpenAI berhasil diambil dari Colab Secrets.")

except userdata.SecretNotFoundError:
    display(Markdown("❌ **Error:** *Secret* 'openai_key' tidak ditemukan. Aplikasi tidak akan berfungsi."))
    OPENAI_API_KEY = None
except ValueError as ve:
    OPENAI_API_KEY = None
except Exception as e:
    display(Markdown(f"🚨 **Error tak terduga saat mengambil kunci API:** {str(e)}"))
    OPENAI_API_KEY = None

# Tahap 1.4: Inisialisasi Klien OpenAI (Sama seperti Tugas 1)
client = None # Inisialisasi client sebagai None dulu
if OPENAI_API_KEY:
    try:
        client = openai.OpenAI(api_key=OPENAI_API_KEY)
        print("🤖 Klien OpenAI berhasil diinisialisasi!")
        display(Markdown("🎉 **Klien OpenAI siap digunakan untuk memproses data tabular.**"))
    except Exception as e:
        display(Markdown(f"🔥 **Error saat inisialisasi klien OpenAI:** {str(e)}"))
else:
    if OPENAI_API_KEY is not None: # Hanya tampilkan jika error bukan karena secret tidak ada
        display(Markdown("🔴 **Klien OpenAI tidak dapat diinisialisasi karena kunci API bermasalah.**"))

if not client:
    display(Markdown("‼️ **PENTING:** Klien OpenAI tidak siap. Fungsi pemrosesan data dengan LLM tidak akan berjalan. Harap periksa pengaturan kunci API Anda."))

✅ Library OpenAI, Pandas, dan pendukung lainnya telah siap!
🔑 Kunci API OpenAI berhasil diambil dari Colab Secrets.
🤖 Klien OpenAI berhasil diinisialisasi!


🎉 **Klien OpenAI siap digunakan untuk memproses data tabular.**

## 💾 Tahap 2: Pemilihan, Pengunggahan, dan Pemuatan Dataset

Pada tahap ini, kita akan fokus pada persiapan data yang akan digunakan untuk diproses oleh Model Bahasa Skala Besar (LLM). Proses ini melibatkan pemilihan dataset yang relevan, mekanisme untuk mengunggah file dataset tersebut ke lingkungan Colab, dan akhirnya memuat data ke dalam struktur yang dapat diolah dengan mudah, yaitu Pandas DataFrame.

### 2.1 Deskripsi Dataset Pilihan
*   **Nama Dataset:** IMDB Dataset of 50K Movie Reviews (Sampel 100 Baris)
*   **Sumber:** Kaggle - [IMDB Dataset of 50K Movie Reviews by lakshmi25npathi](https://www.kaggle.com/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews)
*   **Deskripsi Singkat:** Dataset ini berisi 50.000 ulasan film dari IMDB, masing-masing dengan label sentimen "positive" atau "negative". Untuk efisiensi pengerjaan tugas ini, saya akan menggunakan **sampel yang terdiri dari 100 baris pertama** dari file `IMDB Dataset.csv`.
*   **Kolom Teks yang Akan Diproses:** `review` (berisi teks ulasan film).
*   **Alasan Pemilihan dan Penggunaan Sampel:**
    1.  **Kaya Konten Teks**: Kolom `review` menyediakan ulasan pengguna yang panjang dan mendetail, sangat cocok untuk berbagai tugas NLP menggunakan LLM, seperti peringkasan, analisis sentimen yang lebih mendalam (misalnya, identifikasi emosi spesifik), atau ekstraksi aspek.
    2.  **Struktur Sederhana**: Dataset memiliki struktur kolom yang jelas (`review`, `sentiment`), memudahkan pemrosesan awal.
    3.  **Efisiensi**: Menggunakan sampel 100 baris pertama secara signifikan mengurangi waktu pemrosesan dan biaya API yang terkait dengan penggunaan LLM, sambil tetap menyediakan data yang cukup untuk mendemonstrasikan konsep pemrosesan data tabular. Ini memungkinkan iterasi dan pengujian yang lebih cepat.
    4.  **Relevansi Tugas**: Dataset ulasan sangat umum digunakan dalam aplikasi NLP dan relevan untuk tugas-tugas yang diminta.

*(Saya telah menyiapkan file CSV baru yang berisi 100 baris pertama dari `IMDB Dataset.csv`, dan file inilah yang akan diunggah pada langkah berikutnya.)*

---
Berikutnya adalah kode untuk mengunggah dan memuat dataset tersebut.

In [20]:
# Tahap 2.2: Mengunggah File Dataset (menangani format satu kolom per baris data)
import re
import io
from IPython.display import display, Markdown
import pandas as pd

display(Markdown("📂 **Silakan unggah file dataset Anda (CSV):**"))
uploaded_files = files.upload()

df = None
raw_df_display_limit = 10
lines_to_inspect_on_error = 10
lines_to_print_raw = 5

if not uploaded_files:
    display(Markdown("⚠️ Tidak ada file yang diunggah. Silakan jalankan sel ini lagi dan pilih file."))
else:
    file_name = next(iter(uploaded_files))
    file_content_bytes = uploaded_files[file_name]
    display(Markdown(f"✅ File **'{file_name}'** berhasil diunggah! ({len(file_content_bytes)} bytes)"))

    parsed_data = []
    problematic_lines_examples = []
    processed_line_count = 0
    skipped_line_count = 0

    display(Markdown("Attempting to parse file assuming each data line is a single string containing review and sentiment..."))

    bytes_io_file = io.BytesIO(file_content_bytes)

    first_line_str = ""
    assumed_encoding = 'utf-8'
    decoded_first_line = False
    try:
        first_line_bytes = bytes_io_file.readline()
        encodings_to_try = ['utf-8', 'latin1', 'iso-8859-1']
        for enc in encodings_to_try:
            try:
                first_line_str = first_line_bytes.decode(enc).strip()
                display(Markdown(f"Baris pertama berhasil didekode dengan **{enc}**: '{first_line_str[:100]}...'"))
                assumed_encoding = enc
                decoded_first_line = True
                break
            except UnicodeDecodeError:
                continue
        if not decoded_first_line:
            display(Markdown("❌ **Error:** Tidak dapat mendekode baris pertama file."))
    except Exception as e:
        display(Markdown(f"❌ **Error saat membaca baris pertama:** {e}"))

    header_skipped_flag = False
    line_number_processed_as_data = 0

    if decoded_first_line:
        # Cek header dengan lebih hati-hati, hilangkan semua non-alphanumeric kecuali koma
        # lalu cek apakah sama dengan "review,sentiment"
        cleaned_header_for_check = re.sub(r'[^a-z0-9,]', '', first_line_str.lower())
        if cleaned_header_for_check == "review,sentiment":
            display(Markdown(f"Header ditemukan dan dilewati: '{first_line_str}'"))
            header_skipped_flag = True
        else:
            display(Markdown(f"Baris pertama ('{first_line_str[:50]}...') tidak terdeteksi sebagai header 'review,sentiment'. Akan mencoba memprosesnya sebagai data."))
            bytes_io_file.seek(0) # Kembalikan pointer jika baris pertama bukan header

        raw_lines_printed_count = 0
        display(Markdown(f"\n**Mencetak {lines_to_print_raw} baris data mentah pertama (setelah header jika ada) untuk inspeksi:**"))

        current_line_num_in_file = 0
        for line_bytes in bytes_io_file:
            current_line_num_in_file +=1
            if header_skipped_flag and current_line_num_in_file == 1:
                continue

            line_content_raw_original = ""
            line_content_raw_stripped = ""
            try:
                line_content_raw_original = line_bytes.decode(assumed_encoding)
                line_content_raw_stripped = line_content_raw_original.strip()
            except UnicodeDecodeError:
                if len(problematic_lines_examples) < lines_to_inspect_on_error:
                    problematic_lines_examples.append(f"Baris file ke-{current_line_num_in_file} (Decode Error): {line_bytes[:100]}...")
                skipped_line_count += 1
                continue

            if not line_content_raw_stripped:
                skipped_line_count += 1
                continue

            line_number_processed_as_data += 1

            if raw_lines_printed_count < lines_to_print_raw:
                print(f"RAW DATA Line (file line {current_line_num_in_file}): '{line_content_raw_stripped}'") # Cetak versi stripped
                raw_lines_printed_count += 1

            review_text_candidate = line_content_raw_stripped
            sentiment_label_str = ""

            # --- LOGIKA PARSING BARU ---
            # Kita cari ",positive" atau ",negative" di akhir, lalu bersihkan titik koma

            # Pola untuk mencari ",positive" atau ",negative" yang mungkin diikuti titik koma
            # (,["]?(positive|negative)["]?)((?:;)*)$
            # Grup 1: ,?"positive"? atau ,?"negative"?
            # Grup 2: positive atau negative
            # Grup 3: titik koma
            match = re.search(r'(?i)(,["\']?(positive|negative)["\']?)((?:;)*)$', review_text_candidate)

            if match:
                sentiment_block_with_comma = match.group(1) # Ini akan seperti ',"positive"' atau ',positive'
                sentiment_label_str = match.group(2).lower() # Ini "positive" atau "negative"

                # Teks review adalah semua sebelum blok sentimen (termasuk koma di depannya)
                review_text = review_text_candidate[:-len(match.group(0))] # Hapus seluruh blok yang cocok

                # Bersihkan tanda kutip pembuka/penutup dari review_text jika seluruh review diapit kutip
                if review_text.startswith('"') and review_text.endswith('"'):
                    review_text = review_text[1:-1]

                review_text = review_text.strip()
                review_text = review_text.replace("<br />", " ").replace("<br>", " ").replace("<br/>", " ")
                review_text = review_text.replace("\\n", " ")
                review_text = review_text.replace('""', '"') # Ganti "" menjadi "
                review_text = ' '.join(review_text.split())

                title_text = "" # Kolom title dikosongkan
                parsed_data.append([sentiment_label_str, title_text, review_text])
                processed_line_count += 1
            else:
                if len(problematic_lines_examples) < lines_to_inspect_on_error :
                    problematic_lines_examples.append(f"Baris file ke-{current_line_num_in_file} (No Sentiment Pattern Match at End): {review_text_candidate[:150]}...")
                skipped_line_count += 1

        total_data_lines_attempted = current_line_num_in_file - (1 if header_skipped_flag and cleaned_header_for_check == "review,sentiment" else 0)
        display(Markdown(f"Total baris data yang coba diproses (setelah header jika ada): {total_data_lines_attempted}"))
        display(Markdown(f"Baris yang berhasil diparsing: {processed_line_count}"))
        display(Markdown(f"Baris yang dilewati (error decode/format/no match): {skipped_line_count}"))

        if problematic_lines_examples:
            display(Markdown("**Contoh baris yang bermasalah/dilewati (selain decode error):**"))
            for ex_line in problematic_lines_examples:
                display(Markdown(f"- `{ex_line}`"))

        if parsed_data:
            df = pd.DataFrame(parsed_data, columns=['polarity_str', 'title', 'text'])
            df['polarity'] = df['polarity_str'].map({'positive': 2, 'negative': 1})
            if df['polarity'].isnull().any():
                display(Markdown(f"⚠️ Ditemukan nilai NaN di kolom polarity setelah mapping."))
                df['polarity'] = df['polarity'].fillna(0).astype(int) # Isi NaN dengan 0 atau nilai default lain
            else:
                df['polarity'] = df['polarity'].astype(int)
            df = df[['polarity', 'title', 'text']]
            display(Markdown(f"📄 **DataFrame dibuat dari {len(df)} baris yang berhasil diparsing.**"))
        else:
            display(Markdown("❌ Tidak ada data yang berhasil diparsing menjadi DataFrame."))

    # ... (Sisa kode untuk menampilkan df.head(), df.info(), df.describe() tetap sama) ...
    if df is not None and not df.empty:
        display(Markdown(f"Berikut adalah **{min(raw_df_display_limit, len(df))} baris pertama** dari dataset Anda:"))
        display(df.head(raw_df_display_limit))
        display(Markdown("---"))
        display(Markdown("**Informasi Dataset:**"))
        try:
            buffer = io.StringIO()
            df.info(buf=buffer)
            info_str = buffer.getvalue()
            display(Markdown(f"<pre>{info_str}</pre>"))
        except Exception as info_err:
            display(Markdown(f"Tidak bisa menampilkan df.info(): {info_err}"))

        display(Markdown("---"))
        display(Markdown("**Contoh nilai unik di kolom 'polarity':**"))
        if 'polarity' in df.columns and not df.empty:
            display(df['polarity'].value_counts().head())
        else:
            display(Markdown("Kolom 'polarity' tidak ditemukan atau DataFrame kosong."))

        display(Markdown("---"))
        display(Markdown("**Statistik Deskriptif (untuk kolom numerik):**"))
        numeric_cols = df.select_dtypes(include='number').columns
        if not numeric_cols.empty:
            display(df.describe(include='number'))
        else:
            display(Markdown("Tidak ada kolom numerik untuk dideskripsikan."))

        display(Markdown("---"))
        display(Markdown("**Statistik Deskriptif (untuk kolom objek/kategorikal):**"))
        object_cols = df.select_dtypes(include='object').columns
        if not object_cols.empty:
            display(df.describe(include='object'))
        else:
            display(Markdown("Tidak ada kolom objek untuk dideskripsikan."))
        display(Markdown("---"))
        display(Markdown(f"**Jumlah baris dan kolom:** {df.shape}"))


if df is None or df.empty:
    display(Markdown("‼️ **Dataset tidak berhasil dimuat atau kosong setelah parsing.** Pastikan file Anda valid dan coba lagi."))
elif file_name.endswith('.csv'):
    if df.shape[1] != 3:
        display(Markdown(f"⚠️ **Peringatan Akhir:** Jumlah kolom yang terbaca ({df.shape[1]}) tidak sesuai dengan yang diharapkan (3)."))
    if 'text' in df.columns and df['text'].isnull().sum() > len(df) * 0.1:
        display(Markdown("⚠️ **Peringatan Akhir:** Sebagian nilai di kolom 'text' kosong. Ini mungkin indikasi masalah parsing yang tersisa."))

📂 **Silakan unggah file dataset Anda (CSV):**

Saving REVIEW.CSV to REVIEW (8).CSV


✅ File **'REVIEW (8).CSV'** berhasil diunggah! (130631 bytes)

Attempting to parse file assuming each data line is a single string containing review and sentiment...

Baris pertama berhasil didekode dengan **utf-8**: 'review,sentiment;;;;;;...'

...

---

**Statistik Deskriptif (untuk kolom objek/kategorikal):**

,title,text
count,98,98
unique,1,98
top,,"""A wonderful little production. The filming te..."
freq,98,1


---

**Jumlah baris dan kolom:** (98, 3)

## 🧠 Tahap 3: Mendefinisikan Fungsi untuk Memproses Teks dengan LLM

Setelah data berhasil dimuat, langkah berikutnya adalah membuat fungsi yang akan berinteraksi dengan API LLM untuk memproses setiap potongan teks dari dataset kita. Fungsi `process_text_with_llm` ini akan dirancang untuk menjadi cukup fleksibel:

1.  **Input**:
    *   `text_to_process`: String teks individual yang ingin diolah (misalnya, satu ulasan film).
    *   `user_custom_prompt`: String *prompt* yang akan kita definisikan, memberi instruksi kepada LLM tentang tugas yang harus dilakukan pada `text_to_process`.
    *   `model_name`: Nama model OpenAI yang ingin digunakan (default ke `gpt-3.5-turbo` untuk efisiensi, bisa diganti ke `gpt-4o` untuk kualitas lebih tinggi).
2.  **Proses**:
    *   Menangani kasus di mana `text_to_process` mungkin kosong atau `NaN`.
    *   Membentuk pesan yang akan dikirim ke API, biasanya terdiri dari:
        *   Pesan sistem (opsional, tapi baik untuk mengatur perilaku AI).
        *   Pesan pengguna yang berisi `user_custom_prompt` dan `text_to_process`.
    *   Memanggil API OpenAI.
    *   Menangani potensi error dari API.
3.  **Output**:
    *   String respons yang dihasilkan oleh LLM.
    *   Pesan error jika terjadi masalah.

Fungsi ini akan menjadi inti dari bagaimana kita menerapkan kemampuan LLM pada setiap baris data kita.

In [21]:
# Tahap 3.1: Mendefinisikan Fungsi untuk Memproses Teks dengan LLM

def process_text_with_llm(text_to_process: str, user_custom_prompt: str, model_name: str = "gpt-3.5-turbo"):
    """
    Mengirimkan teks dan prompt kustom ke API OpenAI dan mengembalikan respons LLM.

    Args:
        text_to_process (str): Teks individual yang akan diproses.
        user_custom_prompt (str): Instruksi spesifik untuk LLM tentang apa yang harus dilakukan dengan teks.
                                  Gunakan placeholder seperti {TEXT_PLACEHOLDER} di dalam prompt ini,
                                  yang akan digantikan dengan text_to_process.
        model_name (str, optional): Model OpenAI yang akan digunakan. Default ke "gpt-3.5-turbo".

    Returns:
        str: Respons yang dihasilkan oleh LLM, atau pesan error jika terjadi masalah.
    """
    if not client:
        error_msg = "❌ Klien OpenAI tidak diinisialisasi. Tidak dapat memproses teks."
        display(Markdown(f"<div style='color: red; border: 1px solid red; padding: 10px; border-radius: 5px;'>{error_msg}</div>"))
        return error_msg

    if not text_to_process or pd.isna(text_to_process):
        # display(Markdown("🔸 Input teks kosong atau NaN. Mengembalikan respons default."))
        return "Input teks kosong atau tidak valid."

    # Mengganti placeholder dalam prompt kustom dengan teks aktual
    # Pastikan user_custom_prompt Anda mengandung {TEXT_PLACEHOLDER}
    # Jika tidak, Anda bisa menggabungkannya secara manual.
    # Untuk fleksibilitas, kita asumsikan prompt sudah menyertakan cara menggabungkan teks.
    # Contoh: user_custom_prompt = "Rangkum teks berikut: {TEXT_PLACEHOLDER}"
    # Atau, jika prompt tidak pakai placeholder:
    # final_prompt_content = f"{user_custom_prompt}\n\nTEKS UNTUK DIPROSES:\n\"\"\"\n{text_to_process}\n\"\"\""

    # Kita akan gunakan pendekatan di mana prompt pengguna sudah mencakup cara memasukkan teks.
    # Ini memberi pengguna kontrol lebih besar atas bagaimana teks disajikan ke LLM.
    # Contoh: "Rangkum teks ini: [TEKSNYA DI SINI]" atau "Analisis sentimen dari: [TEKSNYA DI SINI]"
    # Jadi, kita akan menggabungkan prompt dan teks di sini jika prompt tidak memiliki placeholder.
    # Namun, cara terbaik adalah user_custom_prompt sudah siap pakai.
    # Untuk tugas ini, kita akan asumsikan user_custom_prompt adalah instruksi, dan kita gabungkan dengan teks.

    # Pesan sistem untuk memandu AI (bisa disesuaikan atau dihilangkan)
    system_message = "Anda adalah asisten AI yang ahli dalam memproses dan menganalisis teks sesuai instruksi yang diberikan."

    # Gabungkan prompt pengguna dengan teks yang akan diproses
    # Ini adalah cara sederhana. Anda bisa membuat ini lebih canggih
    # dengan placeholder di user_custom_prompt jika diinginkan.
    full_user_message = f"{user_custom_prompt}\n\n---\n\nTeks yang akan diproses:\n```\n{text_to_process}\n```"

    messages_for_api = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": full_user_message}
    ]

    try:
        # print(f"   Mengirim ke model ({model_name}) untuk diproses...") # Debugging jika perlu
        response = client.chat.completions.create(
            model=model_name,
            messages=messages_for_api,
            temperature=0.5, # Sedikit kreativitas, bisa disesuaikan (0.0 untuk deterministik, >0.7 lebih kreatif)
            max_tokens=250  # Batasi panjang respons untuk efisiensi, sesuaikan jika perlu output panjang
        )
        llm_reply = response.choices[0].message.content.strip()
        return llm_reply
    except openai.APIError as e:
        error_detail = f"API Error: {e}"
        print(f"   ❌ Error API saat memproses teks: '{str(text_to_process)[:70]}...': {error_detail}")
        return f"Error API: {str(e)}" # Kembalikan pesan error yang lebih singkat ke DataFrame
    except Exception as e:
        error_detail = f"Unexpected Error: {e}"
        print(f"   ❌ Error tak terduga saat memproses teks: '{str(text_to_process)[:70]}...': {error_detail}")
        return f"Error Tak Terduga: {str(e)}"


# --- Tes Cepat Fungsi (opsional, tidak akan menghasilkan output jika df belum ada) ---
if 'df' in globals() and df is not None and not df.empty and 'text' in df.columns:
    if client: # Hanya jalankan tes jika client ada
        display(Markdown("🧪 **Melakukan tes cepat pada fungsi `process_text_with_llm`...**"))
        sample_text = df['text'].iloc[0] # Ambil teks pertama dari DataFrame
        if pd.isna(sample_text) and len(df) > 1: # Jika teks pertama NaN, coba teks kedua
            sample_text = df['text'].iloc[1]

        if pd.notna(sample_text):
            test_prompt = "Sebutkan topik utama dari teks berikut dalam 3 kata:"
            display(Markdown(f"**Teks Sampel:**\n```\n{sample_text[:200]}...\n```"))
            display(Markdown(f"**Prompt Tes:** `{test_prompt}`"))

            test_response = process_text_with_llm(sample_text, test_prompt)
            display(Markdown(f"**Respons Tes dari LLM:**\n```\n{test_response}\n```"))
        else:
            display(Markdown("Tidak dapat menemukan teks sampel yang valid untuk tes."))
    else:
        display(Markdown("Klien OpenAI tidak siap, tes fungsi dilewati."))
else:
    display(Markdown("DataFrame `df` belum siap atau tidak memiliki kolom 'text', tes fungsi dilewati. "
                     "Pastikan sel Tahap 2 sudah dijalankan dan berhasil memuat data."))

print("\n✅ Fungsi `process_text_with_llm` berhasil didefinisikan.")

🧪 **Melakukan tes cepat pada fungsi `process_text_with_llm`...**

**Teks Sampel:**
```
"A wonderful little production. The filming technique is very unassuming- very old-time-BBC fashion and gives a comforting, and sometimes discomforting, sense of realism to the entire piece. The actor...
```

**Prompt Tes:** `Sebutkan topik utama dari teks berikut dalam 3 kata:`

**Respons Tes dari LLM:**
```
komedi, produksi, realisme
```


✅ Fungsi `process_text_with_llm` berhasil didefinisikan.


## ⚙️ Tahap 4: Iterasi pada DataFrame, Prompting, dan Pemrosesan LLM

Kini saatnya kita benar-benar "melepaskan" LLM pada data kita! Prosesnya adalah sebagai berikut:

1.  **Persiapan**:
    *   Kita akan memastikan DataFrame `df` sudah ada dan valid.
    *   Pengguna akan diminta untuk mengkonfirmasi nama kolom yang berisi teks untuk diproses (misalnya, kolom `review` atau `text`).
    *   Pengguna akan diminta untuk memasukkan **prompt kustom**. Ini adalah instruksi inti yang akan diberikan kepada LLM untuk setiap baris teks. Contohnya:
        *   `"Rangkum ulasan berikut dalam 2 kalimat utama:"`
        *   `"Identifikasi sentimen dari ulasan berikut (jawab hanya dengan: positif, negatif, atau netral):"`
        *   `"Sebutkan tiga kata kunci utama dari teks berikut:"`
    *   Kita akan memilih model LLM yang akan digunakan (misalnya, `gpt-3.5-turbo` atau `gpt-4o`).

2.  **Iterasi dan Pemrosesan**:
    *   Kita akan melakukan iterasi melalui setiap baris dalam DataFrame (atau sampel baris jika dataset sangat besar dan kita hanya ingin menguji).
    *   Untuk setiap baris, teks dari kolom yang dipilih akan diambil.
    *   Fungsi `process_text_with_llm` akan dipanggil dengan teks tersebut dan prompt kustom yang telah ditentukan.
    *   Respons dari LLM akan dikumpulkan.

3.  **Penyimpanan Hasil**:
    *   Semua respons LLM akan disimpan dalam sebuah list.
    *   List ini kemudian akan ditambahkan sebagai kolom baru ke DataFrame kita, misalnya dengan nama `llm_response`.



In [27]:
# Tahap 4.1: Persiapan untuk Iterasi dan Pemrosesan

if 'df' not in globals() or df is None or df.empty:
    display(Markdown("‼️ **Error:** DataFrame `df` tidak ditemukan atau kosong. "
                     "Harap jalankan sel Tahap 2 untuk memuat data terlebih dahulu sebelum melanjutkan."))
else:
    display(Markdown("✅ DataFrame `df` siap untuk diproses."))
    display(Markdown(f"Jumlah baris dalam DataFrame: {len(df)}"))
    display(Markdown(f"Nama kolom yang tersedia: {df.columns.tolist()}"))

    # --- PENGATURAN OLEH PENGGUNA ---
    # 1. Pilih kolom yang berisi teks untuk diproses
    available_text_columns = [col for col in df.columns if df[col].dtype == 'object'] # Ambil kolom string
    if not available_text_columns:
        display(Markdown("‼️ **Error:** Tidak ada kolom teks (object/string) yang ditemukan di DataFrame."))
        # Hentikan eksekusi sel ini jika tidak ada kolom teks
        raise ValueError("Tidak ada kolom teks yang valid di DataFrame untuk diproses.")

    print("\n--- Pengaturan Pemrosesan LLM ---")
    default_text_column = 'text' if 'text' in available_text_columns else available_text_columns[0]
    column_to_process = input(f"Masukkan nama kolom yang berisi teks untuk diproses (pilihan: {available_text_columns}, default: {default_text_column}): ") or default_text_column

    if column_to_process not in df.columns:
        display(Markdown(f"❌ **Error:** Kolom '{column_to_process}' tidak ditemukan di DataFrame. Menggunakan default '{default_text_column}' jika ada, atau kolom teks pertama."))
        column_to_process = default_text_column
        if column_to_process not in df.columns: # Cek lagi jika default juga tidak valid
             column_to_process = available_text_columns[0] # Fallback ke kolom teks pertama
        display(Markdown(f"Akan memproses kolom: **'{column_to_process}'**"))


    # 2. Masukkan prompt kustom Anda
    # Contoh prompt:
    # "Rangkum ulasan berikut dalam satu kalimat singkat:"
    # "Identifikasi sentimen utama dari teks ini (jawab hanya dengan: POSITIF, NEGATIF, atau NETRAL):"
    # "Ekstrak semua entitas nama orang dan organisasi dari teks berikut, pisahkan dengan koma:"
    # "Terjemahkan teks berikut ke Bahasa Inggris:"
    default_user_prompt = "Rangkum teks berikut menjadi satu kalimat inti yang menangkap esensi utamanya:"
    user_custom_prompt_for_llm = input(f"Masukkan prompt kustom Anda untuk LLM (default: '{default_user_prompt}'):\n") or default_user_prompt
    display(Markdown(f"📝 **Prompt yang akan digunakan:** '{user_custom_prompt_for_llm}'"))

    # 3. Pilih model LLM
    # chosen_llm_model = "gpt-4o" # Untuk kualitas terbaik, tapi lebih mahal & lambat
    chosen_llm_model = "gpt-3.5-turbo" # Pilihan default yang baik untuk kecepatan & biaya
    display(Markdown(f"🤖 **Model LLM yang akan digunakan:** '{chosen_llm_model}'"))

    # 4. Tentukan berapa banyak baris yang ingin diproses (untuk pengujian)
    # Set ke len(df) untuk memproses semua baris.
    default_num_rows_to_process = min(10, len(df)) # Proses 10 baris pertama atau semua jika < 10
    try:
        num_rows_to_process_input = input(f"Berapa banyak baris yang ingin Anda proses? (Tekan Enter untuk default: {default_num_rows_to_process} baris, atau ketik 'semua' untuk {len(df)} baris): ")
        if num_rows_to_process_input.lower() == 'semua':
            num_rows_to_process = len(df)
        elif num_rows_to_process_input == "":
            num_rows_to_process = default_num_rows_to_process
        else:
            num_rows_to_process = int(num_rows_to_process_input)
            if num_rows_to_process > len(df):
                display(Markdown(f"Jumlah baris diminta ({num_rows_to_process}) melebihi total baris ({len(df)}). Akan memproses semua {len(df)} baris."))
                num_rows_to_process = len(df)
            elif num_rows_to_process <= 0:
                display(Markdown("Jumlah baris harus positif. Menggunakan default."))
                num_rows_to_process = default_num_rows_to_process
    except ValueError:
        display(Markdown("Input tidak valid untuk jumlah baris. Menggunakan default."))
        num_rows_to_process = default_num_rows_to_process

    display(Markdown(f"⚙️ Akan memproses **{num_rows_to_process}** baris dari kolom **'{column_to_process}'**."))

    # --- ITERASI DAN PEMROSESAN ---
    llm_responses = []
    display(Markdown("\n🚀 **Memulai pemrosesan baris dengan LLM...** (Ini mungkin memakan waktu)"))

    # Ambil subset DataFrame yang akan diproses
    df_subset_to_process = df.head(num_rows_to_process)

    for index, row in df_subset_to_process.iterrows():
        text_from_row = str(row[column_to_process]) # Pastikan dikonversi ke string

        display(Markdown(f"\n--- Memproses Baris Indeks: {index} ---"))
        # display(Markdown(f"Teks Asli (awal): '{text_from_row[:100]}...'")) # Komentari jika terlalu verbose

        if not text_from_row.strip() or pd.isna(text_from_row):
            display(Markdown("SKIP: Teks kosong atau NaN."))
            llm_responses.append("Input teks kosong/NaN") # Tambahkan placeholder untuk menjaga panjang list
            continue

        response_from_llm = process_text_with_llm(text_from_row, user_custom_prompt_for_llm, model_name=chosen_llm_model)
        llm_responses.append(response_from_llm)

        # Tampilkan progres sederhana
        display(Markdown(f"🤖 Respons LLM (awal): '{str(response_from_llm)[:150]}...'"))

    display(Markdown("\n🏁 **Pemrosesan LLM Selesai!**"))

    # --- MENAMBAHKAN HASIL KE DATAFRAME ---
    # Pastikan panjang llm_responses sesuai dengan jumlah baris yang diproses
    if len(llm_responses) == num_rows_to_process:
        # Jika memproses subset, kita perlu cara untuk menambahkan ini kembali ke df utama
        # atau membuat df baru. Untuk sekarang, kita tambahkan ke subset yang diproses.
        # Membuat nama kolom baru yang unik, misal berdasarkan prompt
        prompt_slug = "".join(filter(str.isalnum, user_custom_prompt_for_llm))[:20] # Ambil 20 karakter alfanumerik dari prompt
        new_column_name = f"llm_output_{prompt_slug}"

        # Jika kita hanya memproses subset (df_subset_to_process),
        # kita bisa membuat kolom baru di df_subset_to_process
        # atau jika ingin di df utama, perlu penanganan indeks yang hati-hati.
        # Untuk kesederhanaan, kita akan bekerja dengan df_subset_to_process jika num_rows_to_process < len(df)
        # atau langsung ke df jika memproses semua.

        if num_rows_to_process < len(df):
            # Buat salinan dari subset untuk dimodifikasi
            df_processed_subset = df_subset_to_process.copy()
            df_processed_subset[new_column_name] = llm_responses
            display(Markdown(f"\n✅ Respons LLM telah ditambahkan sebagai kolom **'{new_column_name}'** pada **subset DataFrame** yang diproses."))
            # Untuk melihat hasilnya, kita akan display df_processed_subset di tahap berikutnya
            # Atau, jika ingin menggabungkan kembali ke df utama:
            # df.loc[df_subset_to_process.index, new_column_name] = llm_responses
            # display(Markdown(f"\n✅ Respons LLM telah ditambahkan sebagai kolom **'{new_column_name}'** pada DataFrame utama untuk baris yang diproses."))
        else: # Memproses semua baris
            df[new_column_name] = llm_responses
            display(Markdown(f"\n✅ Respons LLM telah ditambahkan sebagai kolom **'{new_column_name}'** pada DataFrame."))
            df_processed_subset = df # Agar variabelnya konsisten untuk tahap berikutnya

    else:
        display(Markdown(f"⚠️ **Peringatan:** Jumlah respons LLM ({len(llm_responses)}) tidak sesuai dengan jumlah baris yang diproses ({num_rows_to_process}). Kolom baru tidak ditambahkan."))

✅ DataFrame `df` siap untuk diproses.

Jumlah baris dalam DataFrame: 98

Nama kolom yang tersedia: ['polarity', 'title', 'text']


--- Pengaturan Pemrosesan LLM ---
Masukkan nama kolom yang berisi teks untuk diproses (pilihan: ['title', 'text'], default: text): text
Masukkan prompt kustom Anda untuk LLM (default: 'Rangkum teks berikut menjadi satu kalimat inti yang menangkap esensi utamanya:'):
rangkumlah 10 baris pertama, menjadi 1 kalimat


📝 **Prompt yang akan digunakan:** 'rangkumlah 10 baris pertama, menjadi 1 kalimat'

...

🤖 Respons LLM (awal): 'Jika Anda menyukai tawa yang menggugah perut dan asli, Anda akan menyukai film ini, baik Anda muda atau tua maka Anda akan menyukai film ini, bahkan i...'


--- Memproses Baris Indeks: 9 ---

🤖 Respons LLM (awal): '"Phil the Alien" is a quirky film with humor based on oddness rather than punchlines, initially amusing but losing its charm as the movie progresses, ...'


🏁 **Pemrosesan LLM Selesai!**


✅ Respons LLM telah ditambahkan sebagai kolom **'llm_output_rangkumlah10barisper'** pada **subset DataFrame** yang diproses.

## 📊 Tahap 5: Menampilkan Hasil Pemrosesan LLM

Setelah LLM selesai memproses baris-baris data yang dipilih, responsnya telah ditambahkan sebagai kolom baru ke dalam DataFrame (atau subset DataFrame jika tidak semua baris diproses).

Sekarang, kita akan menampilkan beberapa baris dari DataFrame yang telah diperbarui ini. Kita akan fokus pada kolom teks asli yang diproses dan kolom baru yang berisi output dari LLM agar mudah membandingkan dan melihat hasilnya.

Jika Anda memproses subset data, kita akan menampilkan `df_processed_subset`. Jika Anda memproses seluruh DataFrame, kita akan menampilkan `df` yang sudah dimodifikasi.

In [28]:
# Tahap 5.1: Menampilkan DataFrame dengan Hasil LLM

if 'df_processed_subset' in globals() and df_processed_subset is not None and not df_processed_subset.empty:
    display(Markdown(f"✨ **Menampilkan Hasil Pemrosesan untuk {len(df_processed_subset)} Baris** ✨"))

    # Kolom yang ingin ditampilkan: kolom teks asli dan kolom hasil LLM
    # Kita perlu nama kolom hasil LLM yang baru saja dibuat.
    # Kita bisa ambil nama kolom terakhir jika itu adalah kolom LLM,
    # atau kita bisa gunakan variabel new_column_name dari sel sebelumnya jika masih dalam scope.
    # Untuk lebih aman, kita cari kolom yang namanya diawali "llm_output_"

    llm_result_column_name = None
    if 'new_column_name' in globals() and new_column_name in df_processed_subset.columns:
        llm_result_column_name = new_column_name
    else: # Fallback jika new_column_name tidak ada di scope ini (misal sel dijalankan terpisah)
        for col in reversed(df_processed_subset.columns): # Cari dari belakang
            if col.startswith("llm_output_"):
                llm_result_column_name = col
                break

    if llm_result_column_name and column_to_process in df_processed_subset.columns:
        display(Markdown(f"Kolom yang ditampilkan: '{column_to_process}' (asli) dan '{llm_result_column_name}' (hasil LLM)."))

        # Mengatur opsi tampilan Pandas agar teks panjang tidak terpotong saat ditampilkan
        pd.set_option('display.max_colwidth', None) # Tampilkan isi kolom penuh
        # pd.set_option('display.max_rows', None) # Hati-hati jika datanya banyak

        # Menampilkan kolom yang relevan
        # Jika Anda juga ingin melihat 'polarity', tambahkan ke list di bawah
        columns_to_display = [column_to_process, llm_result_column_name]
        if 'polarity' in df_processed_subset.columns:
            columns_to_display.insert(0, 'polarity') # Tambahkan polarity di awal jika ada

        display(df_processed_subset[columns_to_display].head(num_rows_to_process)) # Tampilkan sejumlah baris yang diproses

        # Kembalikan opsi display ke default jika perlu (opsional)
        # pd.reset_option('display.max_colwidth')
        # pd.reset_option('display.max_rows')

    elif column_to_process in df_processed_subset.columns:
        display(Markdown(f"Tidak dapat menemukan kolom hasil LLM secara otomatis. Menampilkan kolom '{column_to_process}' saja."))
        display(df_processed_subset[[column_to_process]].head(num_rows_to_process))
    else:
        display(Markdown("⚠️ Tidak dapat menampilkan hasil karena kolom teks asli atau kolom hasil LLM tidak ditemukan di subset yang diproses."))
        display(df_processed_subset.head(num_rows_to_process))

elif 'df' in globals() and df is not None and not df.empty and any(col.startswith("llm_output_") for col in df.columns):
    # Ini terjadi jika semua baris diproses dan kolom ditambahkan langsung ke 'df'
    display(Markdown(f"✨ **Menampilkan Hasil Pemrosesan untuk Seluruh DataFrame ({len(df)} Baris)** ✨"))

    llm_result_column_name = None
    if 'new_column_name' in globals() and new_column_name in df.columns:
        llm_result_column_name = new_column_name
    else:
        for col in reversed(df.columns):
            if col.startswith("llm_output_"):
                llm_result_column_name = col
                break

    if llm_result_column_name and column_to_process in df.columns:
        display(Markdown(f"Kolom yang ditampilkan: '{column_to_process}' (asli) dan '{llm_result_column_name}' (hasil LLM)."))
        pd.set_option('display.max_colwidth', None)

        columns_to_display = [column_to_process, llm_result_column_name]
        if 'polarity' in df.columns:
            columns_to_display.insert(0, 'polarity')

        display(df[columns_to_display].head(num_rows_to_process)) # Tampilkan sejumlah baris yang diproses (atau semua jika num_rows_to_process = len(df))
    elif column_to_process in df.columns:
        display(Markdown(f"Tidak dapat menemukan kolom hasil LLM secara otomatis. Menampilkan kolom '{column_to_process}' saja."))
        display(df[[column_to_process]].head(num_rows_to_process))
    else:
        display(Markdown("⚠️ Tidak dapat menampilkan hasil karena kolom teks asli atau kolom hasil LLM tidak ditemukan di DataFrame."))
        display(df.head(num_rows_to_process))
else:
    display(Markdown("‼️ **Tidak ada data yang diproses atau DataFrame hasil tidak ditemukan.** "
                     "Pastikan Tahap 4 sudah dijalankan dan berhasil menambahkan kolom hasil LLM."))

✨ **Menampilkan Hasil Pemrosesan untuk 10 Baris** ✨

Kolom yang ditampilkan: 'text' (asli) dan 'llm_output_rangkumlah10barisper' (hasil LLM).

,polarity,text,llm_output_rangkumlah10barisper
0,2,"""A wonderful little production. The filming technique is very unassuming- very old-time-BBC fashion and gives a comforting, and sometimes discomforting, sense of realism to the entire piece. The actors are extremely well chosen- Michael Sheen not only ""has got all the polari"" but he has all the voices down pat too! You can truly see the seamless editing guided by the references to Williams' diary entries, not only is it well worth the watching but it is a terrificly written and performed piece. A masterful production about one of the great master's of comedy and his life. The realism really comes home with the little things: the fantasy of the guard which, rather than use the traditional 'dream' techniques remains solid then disappears. It plays on our knowledge and our senses, particularly with the scenes concerning Orton and Halliwell and the sets (particularly of their flat with Halliwell's murals decorating every surface) are terribly well done.","""A wonderful little production with unassuming filming technique in old-time-BBC fashion, providing a comforting and sometimes discomforting sense of realism, featuring well-chosen actors like Michael Sheen showcasing seamless editing guided by references to Williams' diary entries, resulting in a terrifically written and performed piece about one of the great masters of comedy and his life, where realism shines through in the little details and the sets, particularly in Orton and Halliwell's flat with Halliwell's murals decorating every surface, making it well worth watching."""
1,2,"""I thought this was a wonderful way to spend time on a too hot summer weekend, sitting in the air conditioned theater and watching a light-hearted comedy. The plot is simplistic, but the dialogue is witty and the characters are likable (even the well bread suspected serial killer). While some may be disappointed when they realize this is not Match Point 2: Risk Addiction, I thought it was proof that Woody Allen is still fully in control of the style many of us have grown to love. This was the most I'd laughed at one of Woody's comedies in years (dare I say a decade?). While I've never been impressed with Scarlet Johanson, in this she managed to tone down her ""sexy"" image and jumped right into a average, but spirited young woman. This may not be the crown jewel of his career, but it was wittier than ""Devil Wears Prada"" and more interesting than ""Superman"" a great comedy to go see with friends.","Saya pikir ini adalah cara yang indah untuk menghabiskan waktu di akhir pekan musim panas yang terlalu panas, duduk di teater ber-AC dan menonton komedi yang ringan dengan plot yang sederhana namun dialog yang cerdas dan karakter yang menyenangkan (bahkan tersangka pembunuh berantai yang berpendidikan baik)."
2,1,"Basically there's a family where a little boy (Jake) thinks there's a zombie in his closet & his parents are fighting all the time. This movie is slower than a soap opera... and suddenly, Jake decides to become Rambo and kill the zombie. OK, first of all when you're going to make a film you must Decide if its a thriller or a drama! As a drama the movie is watchable. Parents are divorcing & arguing like in real life. And then we have Jake with his closet which totally ruins all the film! I expected to see a BOOGEYMAN similar movie, and instead i watched a drama with some meaningless thriller spots. 3 out of 10 just for the well playing parents & descent dialogs. As for the shots with Jake: just ignore them.","Basically, there's a family where a little boy (Jake) believes there's a zombie in his closet, his parents are constantly fighting, the movie is slower than a soap opera, and suddenly Jake decides to become Rambo and kill the zombie, leading to a confusing mix of thriller and drama elements."
3,2,"""Petter Mattei's ""Love in the Time of Money"" is a visually stunning film to watch. Mr. Mattei offers us a vivid portrait about human re

## 📝 Tahap 6: Penjelasan, Pilihan Desain, dan Analisis Hasil (Tugas 2)

### Deskripsi Tugas yang Dilakukan
Pada Tugas 2 ini, tujuan utamanya adalah menerapkan kemampuan Model Bahasa Skala Besar (LLM) untuk memproses data tabular secara programatik. Secara spesifik, aplikasi ini melakukan hal berikut:
1.  Memuat dataset tabular yang berisi ulasan teks (dalam kasus ini, sampel dari dataset ulasan film IMDB).
2.  Memungkinkan pengguna untuk memilih kolom teks yang akan diproses.
3.  Memungkinkan pengguna untuk mendefinisikan *prompt* kustom yang akan diterapkan pada setiap baris teks yang dipilih.
4.  Melakukan iterasi pada sejumlah baris data yang ditentukan pengguna.
5.  Untuk setiap baris, mengirimkan teks ulasan dan *prompt* kustom ke API OpenAI (menggunakan model yang dipilih, misalnya `gpt-3.5-turbo`).
6.  Menerima respons dari LLM dan menyimpannya sebagai kolom baru dalam DataFrame.
7.  Menampilkan DataFrame hasil yang memuat teks asli dan output dari LLM untuk perbandingan.

### Informasi Dataset yang Digunakan
*   **Nama Dataset:** IMDB Dataset of 50K Movie Reviews (Sampel 100 Baris Pertama)
*   **Sumber:** Kaggle - [IMDB Dataset of 50K Movie Reviews by lakshmi25npathi](https://www.kaggle.com/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews)
*   **Deskripsi Singkat:** Dataset ini berisi ulasan film dari IMDB, masing-masing dengan label sentimen "positive" atau "negative". Untuk tugas ini, digunakan sampel 100 baris pertama untuk efisiensi.
*   **Kolom Teks yang Diproses:** `text` (berisi teks ulasan film).
    *   *(Catatan: Pada eksekusi terakhir, saya menggunakan kolom `text` dari dataset REVIEW.CSV yang berisi ulasan umum, bukan spesifik IMDB, dan berhasil memproses 98 baris data setelah header dilewati).*
*   **Alasan Pemilihan:** Dataset ulasan teks kaya akan informasi subjektif dan naratif, menjadikannya kandidat yang baik untuk tugas-tugas seperti peringkasan, analisis sentimen, atau ekstraksi informasi menggunakan LLM. Penggunaan sampel data yang lebih kecil (100 baris, dan diproses 10 baris dalam contoh terakhir) memungkinkan iterasi dan pengujian yang cepat tanpa biaya API yang besar.

### Pilihan Desain dan Implementasi
1.  **Pemuatan Data Fleksibel**: Kode dirancang untuk mencoba mem-parse file CSV dengan berbagai strategi, termasuk penanganan header dan pemisahan manual baris demi baris jika formatnya tidak standar. Ini terbukti penting dalam mengatasi tantangan format file `REVIEW.CSV` yang awalnya sulit diparsing.
2.  **Fungsi Pemrosesan LLM Generik (`process_text_with_llm`)**:
    *   Fungsi ini dibuat untuk menerima teks apa pun dan *prompt* kustom, memberikan fleksibilitas untuk berbagai jenis tugas pemrosesan teks.
    *   Model yang digunakan adalah `gpt-3.5-turbo` untuk keseimbangan antara biaya, kecepatan, dan kemampuan. Parameter seperti `temperature` dan `max_tokens` diatur untuk respons yang cukup konsisten dan tidak terlalu panjang.
3.  **Interaksi Pengguna untuk Konfigurasi**:
    *   Pengguna dapat memilih kolom teks yang akan diproses.
    *   Pengguna dapat memasukkan *prompt* kustom mereka sendiri, memungkinkan eksperimen dengan berbagai instruksi untuk LLM.
    *   Pengguna dapat menentukan jumlah baris yang akan diproses, sangat berguna untuk *debugging* dan pengujian awal pada sampel kecil.
4.  **Penambahan Hasil ke DataFrame**: Respons dari LLM untuk setiap baris ditambahkan sebagai kolom baru ke DataFrame (atau subsetnya), memudahkan analisis dan perbandingan hasil. Nama kolom baru dibuat secara dinamis berdasarkan *prompt* untuk menghindari tumpang tindih jika proses dijalankan beberapa kali dengan *prompt* berbeda.

### Prompt yang Digunakan dan Analisis Singkat Hasil
Pada eksekusi terakhir, *prompt* yang digunakan adalah:
`"rangkumlah 10 baris pertama, menjadi 1 kalimat"`

Dan jumlah baris yang diproses adalah 10.

**Observasi Hasil:**
*   LLM (gpt-3.5-turbo) berhasil menghasilkan rangkuman satu kalimat untuk **setiap** dari 10 ulasan yang diproses. Ini sesuai dengan bagaimana *prompt* diterapkan secara individual per baris.
*   Kualitas rangkuman bervariasi. Beberapa rangkuman cukup baik dalam menangkap esensi ulasan, sementara yang lain mungkin terlalu umum atau kehilangan beberapa detail penting, yang wajar untuk model sekelas `gpt-3.5-turbo` dengan *prompt* yang relatif sederhana dan batasan satu kalimat.
*   Menariknya, beberapa respons rangkuman dihasilkan dalam Bahasa Indonesia, meskipun mayoritas ulasan asli dalam Bahasa Inggris. Ini bisa disebabkan oleh:
    *   *Prompt* yang saya berikan (`rangkumlah 10 baris pertama, menjadi 1 kalimat`) mengandung kata-kata dalam Bahasa Indonesia. LLM mungkin mencoba menyesuaikan bahasa outputnya dengan bahasa *prompt*.
    *   Karakteristik model `gpt-3.5-turbo` yang terkadang bisa "beralih" bahasa jika tidak ada instruksi bahasa output yang eksplisit.
*   Kolom `title` sengaja dikosongkan dalam proses parsing karena format file `REVIEW.CSV` tidak menyediakan pemisah yang jelas untuk judul. Fokus utama adalah pada pemrosesan kolom teks ulasan utama.

**Potensi Peningkatan:**
*   **Prompt Engineering Lebih Lanjut**: Untuk hasil yang lebih spesifik atau berkualitas lebih tinggi, *prompt* bisa dibuat lebih detail, misalnya dengan menyertakan contoh (*few-shot prompting*), atau meminta output dalam format tertentu (misalnya, JSON).
*   **Instruksi Bahasa**: Jika output dalam bahasa tertentu diinginkan secara konsisten, instruksi bahasa (misalnya, "Rangkum dalam Bahasa Inggris:") harus ditambahkan ke *prompt*.
*   **Pemrosesan Batch (jika API mendukung)**: Untuk dataset yang sangat besar, jika API mendukung *batch processing*, itu bisa lebih efisien daripada panggilan API sekuensial per baris.
*   **Ekstraksi Judul (jika memungkinkan)**: Jika ada pola untuk judul dalam data asli, logika parsing bisa ditingkatkan untuk mengekstraknya.

### Kesimpulan Tugas 2
Tugas ini berhasil mendemonstrasikan bagaimana LLM dapat digunakan secara programatik untuk melakukan tugas pemrosesan teks pada setiap baris data tabular. Proses ini melibatkan tantangan dalam parsing data dengan format yang tidak standar, pentingnya *prompt engineering* yang cermat, dan kemampuan untuk mengiterasi serta mengintegrasikan hasil LLM kembali ke dalam struktur data tabular. Aplikasi ini menyediakan dasar untuk otomatisasi berbagai tugas analisis teks pada skala yang lebih besar.



## 💡 Tahap Bonus C: Demonstrasi dan Penjelasan Detail Fitur Bonus Multi-Modal

Fitur bonus yang diimplementasikan adalah **Interaksi Multi-Modal**, di mana pengguna dapat menyertakan gambar bersama dengan kueri teks mereka, dan LLM akan memproses keduanya untuk memberikan respons.

### B.2 Implementasi Teknis dan Integrasi (Ringkasan)
Untuk mengaktifkan kemampuan multi-modal ini, beberapa modifikasi dan penambahan dilakukan pada kode aplikasi chat Tugas 1:

1.  **Pemilihan Model LLM**: Model yang digunakan dialihkan ke `gpt-4o`, yang secara native mendukung input teks dan gambar. Ini diatur dalam variabel `chosen_model` pada loop interaktif.
2.  **Fungsi Encoding Gambar**: Sebuah fungsi baru, `image_to_base64(image_bytes, mime_type)`, ditambahkan untuk mengonversi data byte gambar yang diunggah pengguna menjadi format data URI base64. Format ini adalah salah satu cara standar untuk mengirim data gambar ke API OpenAI. Fungsi ini juga mencoba menangani tipe MIME gambar.
3.  **Modifikasi Fungsi `chat_with_ai_multimodal`**:
    *   Fungsi inti `chat_with_ai` dimodifikasi (atau dibuat versi baru bernama `chat_with_ai_multimodal`) untuk menerima argumen opsional `image_base64_data_uri`.
    *   Struktur pesan (`messages`) yang dikirim ke API OpenAI diubah. Jika ada input gambar, pesan pengguna (`user_message_content_parts`) akan menjadi sebuah *list* yang berisi dua *item*: satu bertipe `text` untuk kueri teks, dan satu lagi bertipe `image_url` yang berisi data URI base64 gambar.
    *   Parameter `max_tokens` pada panggilan API mungkin sedikit dinaikkan (misalnya ke 500) untuk mengakomodasi respons yang mungkin lebih panjang ketika mendeskripsikan atau menganalisis gambar.
4.  **Modifikasi Loop Interaktif Utama**:
    *   Setelah pengguna memasukkan teks, program bertanya apakah pengguna ingin menyertakan gambar (`y/n`).
    *   Jika ya, `google.colab.files.upload()` digunakan untuk memungkinkan pengguna mengunggah file gambar (PNG, JPG, GIF, WEBP).
    *   Byte gambar yang diunggah kemudian dikonversi ke data URI base64 menggunakan fungsi `image_to_base64`.
    *   Data URI gambar ini (jika ada) kemudian diteruskan ke fungsi `chat_with_ai_multimodal` bersama dengan input teks dan riwayat percakapan.
    *   Ada konfirmasi visual sederhana (`🖼️ (Gambar disertakan dalam permintaan ke AI)`) yang ditampilkan kepada pengguna.

### B.3 Demonstrasi Fungsionalitas
Berikut adalah contoh log interaksi yang menunjukkan fitur multi-modal beraksi. Saya mengunggah gambar Monumen Nasional (Monas) dan mengajukan pertanyaan terkait gambar tersebut:

```text
🚀 Aplikasi Chat Multi-Modal Siap Dimulai! 🚀
Ketik pesan Anda. Jika ingin menyertakan gambar, jawab 'y' saat ditanya.
Ketik 'exit' atau 'quit' untuk keluar.
--------------------------------------------------
ℹ️ Menggunakan model: *gpt-4o*

👤 Anda (teks): tolong jelaskan gambar apa ini
🖼️ Ingin menyertakan gambar dengan pesan ini? (y/n, default: n): y
📂 Silakan unggah satu file gambar (PNG, JPG, GIF, WEBP):
[Pengguna mengunggah file 8437415341664213099.jpg]
👍 Gambar '8437415341664213099.jpg' (tipe: image/jpeg) berhasil diunggah.

👤 Anda: tolong jelaskan gambar apa ini
🖼️ (Gambar disertakan dalam permintaan ke AI)
🤖 AI (gpt-4o): Gambar ini menunjukkan Monumen Nasional (Monas) yang terletak di Jakarta, Indonesia. Monas merupakan ikon ibu kota yang melambangkan perjuangan kemerdekaan Indonesia. Monumen ini memiliki struktur yang tinggi dengan puncak yang berbentuk lidah api berlapis emas. Di sekitarnya, terlihat latar gedung-gedung tinggi pada malam hari.

👤 Anda (teks): tolong jelaskan sejarah gambar berikut
🖼️ Ingin menyertakan gambar dengan pesan ini? (y/n, default: n): y
📂 Silakan unggah satu file gambar (PNG, JPG, GIF, WEBP):
[Pengguna mengunggah file 8437415341664213099.jpg lagi]
👍 Gambar '8437415341664213099 (1).jpg' (tipe: image/jpeg) berhasil diunggah.

👤 Anda: tolong jelaskan sejarah gambar berikut
🖼️ (Gambar disertakan dalam permintaan ke AI)
🤖 AI (gpt-4o): Monumen Nasional (Monas) dibangun untuk memperingati perjuangan rakyat Indonesia dalam mencapai kemerdekaan. Peletakan batu pertama dilakukan pada 17 Agustus 1961, di bawah pemerintahan Presiden Soekarno. Monas dirancang oleh arsitek Friedrich Silaban dan R.M. Soedarsono.

Monas memiliki tinggi 132 meter dan puncaknya dihiasi oleh lidah api berlapis emas. Monumen ini diresmikan pada 12 Juli 1975. Di dalamnya terdapat museum sejarah perjuangan kemerdekaan Indonesia, dan di bagian atasnya terdapat dek observasi yang menawarkan pemandangan Kota Jakarta.

Monas juga menjadi lokasi berbagai acara penting dan peringatan nasional. Monumen ini merupakan simbol semangat kemerdekaan dan kebanggaan nasional bagi rakyat Indonesia.

👤 Anda (teks): exit
👋 Terima kasih sudah mengobrol! Sampai jumpa lagi!
--------------------------------------------------
🏁 Sesi Chat Multi-Modal Selesai.